# Dataset Cleaning 

The main objectives of this notebook are: 
- Resolve data quality issues 
- Encode the categorical features 
- Build a preprocessing pipeline

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import polars as pl 
from google.cloud import bigquery
import numpy as np 
import sys 
import os 


# Manually add path to read from another folder 
path2add = os.path.normpath(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), os.path.pardir, "utils"))
)
if not (path2add in sys.path):
    sys.path.append(path2add)

from preprocessing import *

In [3]:
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [4]:
# df = pl.read_csv("../data/google_analytics_data.csv",ignore_errors=True)

In [5]:
# Create client object
client = bigquery.Client()

In [6]:
QUERY = ("SELECT `date`, `visitStartTime`, `fullVisitorId`, `geoNetwork`.`subContinent`, `geoNetwork`.`country`,"
        "`geoNetwork`.`city`,`device`.`browser`, `device`.`operatingSystem`,`device`.`deviceCategory`,"
        "`trafficSource`.`source`, `h`.`item`.`transactionId`,`hp`.`v2ProductName`,`hp`.`v2ProductCategory` "
        "FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170731`, UNNEST(hits) AS h,"
        "UNNEST(h.product) AS hp;")

In [7]:
run_query = client.query(QUERY)

In [8]:
df = pl.from_arrow(run_query.to_arrow())

In [9]:
df.head()

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,i64,str,str,str,str,str,str,str,str,str,str,str
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Lunch Bag""","""Home/Bags/More Bags/"""
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Electronics Accessory Pouch""","""Home/Bags/More Bags/"""
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Sport Bag""","""Home/Bags/More Bags/"""
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Slim Utility Travel Bag""","""Home/Bags/More Bags/"""
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Suitcase Organizer Cubes""","""Home/Bags/More Bags/"""


In [10]:
df.shape

(49512, 13)

In [11]:
df.columns

['date',
 'visitStartTime',
 'fullVisitorId',
 'subContinent',
 'country',
 'city',
 'browser',
 'operatingSystem',
 'deviceCategory',
 'source',
 'transactionId',
 'v2ProductName',
 'v2ProductCategory']

In [12]:
df.sample()

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,i64,str,str,str,str,str,str,str,str,str,str,str
"""20170731""",1501559192,"""5458293734660497633""","""Northern America""","""Canada""","""not available in demo dataset""","""Chrome""","""Windows""","""desktop""","""google""",null,"""YouTube Women's Short Sleeve T…","""Home/Shop by Brand/YouTube/"""


In [13]:
df.describe()

statistic,date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,str,f64,str,str,str,str,str,str,str,str,str,str,str
"""count""","""49512""",49512.0,"""49512""","""49512""","""49512""","""49512""","""49512""","""49512""","""49512""","""49512""","""462""","""49512""","""49512"""
"""null_count""","""0""",0.0,"""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""49050""","""0""","""0"""
"""mean""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""std""",null,21502.167394,null,null,null,null,null,null,null,null,null,null,null
"""min""","""20170731""",1.5015e9,"""0002457163364254438""","""Australasia""","""Algeria""","""(not set)""","""Amazon Silk""","""(not set)""","""desktop""","""(direct)""","""ORD201707311786""","""1 oz Hand Sanitizer""","""${escCatTitle}"""
"""25%""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""50%""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""75%""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""max""","""20170731""",1.5016e9,"""9997362993085245352""","""Western Europe""","""Vietnam""","""not available in demo dataset""","""YaBrowser""","""iOS""","""tablet""","""youtube.com""","""ORD201707312663""","""YouTube Youth Short Sleeve Tee…","""Wearables/Men's T-Shirts/"""


In [14]:
df.filter(pl.col('transactionId') != 'null')

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,i64,str,str,str,str,str,str,str,str,str,str,str
"""20170731""",1501525374,"""186231215995844689""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707311786""","""Google Men's Bike Short Sleeve…","""Apparel"""
"""20170731""",1501525374,"""186231215995844689""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707311786""","""Google Men's Bike Short Sleeve…","""Apparel"""
"""20170731""",1501549028,"""0345672699449577691""","""Northern America""","""United States""","""Kirkland""","""Chrome""","""Chrome OS""","""desktop""","""(direct)""","""ORD201707312107""","""Google Blackout Cap""","""Headgear"""
"""20170731""",1501549028,"""0345672699449577691""","""Northern America""","""United States""","""Kirkland""","""Chrome""","""Chrome OS""","""desktop""","""(direct)""","""ORD201707312107""","""Google Women's Vintage Hero Te…","""Apparel"""
"""20170731""",1501549028,"""0345672699449577691""","""Northern America""","""United States""","""Kirkland""","""Chrome""","""Chrome OS""","""desktop""","""(direct)""","""ORD201707312107""","""Google Men's Long & Lean Tee C…","""Apparel"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""20170731""",1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707312663""","""Google 25 oz Red Stainless Ste…","""Drinkware"""
"""20170731""",1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707312663""","""Android 17oz Stainless Steel S…","""Drinkware"""
"""20170731""",1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707312663""","""Android Men's Short Sleeve Her…","""Apparel"""


In [15]:
# df = df.with_columns(pl.col('date').str.strptime(pl.Date, format="%Y%m%d"))
# df.head()

In [16]:
df = df.filter(~pl.col('v2ProductCategory').str.contains('origCatName'))
df = df.with_columns(pl.col("fullVisitorId").cast(str))
df.write_csv("../data/google_analytics_data.csv")

# Encoding the df

- The data will now be encoded using target encoding
    - It is an effective technique used for high cardinality data, such as our dataset 

In [17]:
encoded_df = target_encoding(df)
encoded_df.write_csv('../data/encoded_df.csv')
encoded_df

fullVisitorId,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductCategory
str,f64,f64,f64,f64,f64,f64,i32,f64
"""2219384770970157334""",0.0,0.005886,0.001795,0.00545,0.01104,0.003369,0,0.0
"""2219384770970157334""",0.0,0.005886,0.001795,0.00545,0.01104,0.003369,0,0.0
"""2219384770970157334""",0.0,0.005886,0.001795,0.00545,0.01104,0.003369,0,0.0
"""2219384770970157334""",0.0,0.005886,0.001795,0.00545,0.01104,0.003369,0,0.0
"""2219384770970157334""",0.0,0.005886,0.001795,0.00545,0.01104,0.003369,0,0.0
…,…,…,…,…,…,…,…,…
"""7483600664917507409""",0.013646,0.012644,0.011229,0.016248,0.01104,0.019489,1,0.254777
"""7483600664917507409""",0.013646,0.012644,0.011229,0.016248,0.01104,0.019489,1,0.262
"""7483600664917507409""",0.013646,0.012644,0.011229,0.016248,0.01104,0.019489,1,0.262


In [18]:
# encoded_df = label_encoding(df)
# encoded_df

- User profiles are created to calculate the similarity between past visitors and the current visitor, to be used to create recommendations

In [19]:
user_profiles_ = user_profiles(encoded_df)
user_profiles_.head()

fullVisitorId,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductCategory
str,list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[i32],list[f64]
"""4232939187808568604""",[0.013646],[0.0],[0.000797],[0.000566],[0.003792],[0.003369],[0],[0.0]
"""8996390266519917614""","[0.013646, 0.013646, … 0.013646]","[0.003949, 0.003949, … 0.003949]","[0.011229, 0.011229, … 0.011229]","[0.00545, 0.00545, … 0.00545]","[0.01104, 0.01104, … 0.01104]","[0.019489, 0.019489, … 0.019489]","[0, 0, … 0]","[0.0, 0.0, … 0.25]"
"""845742636528363096""","[0.0, 0.0, … 0.0]","[0.005886, 0.005886, … 0.005886]","[0.000797, 0.000797, … 0.000797]","[0.000566, 0.000566, … 0.000566]","[0.003792, 0.003792, … 0.003792]","[0.003369, 0.003369, … 0.003369]","[0, 0, … 0]","[0.0, 0.0, … 0.0]"
"""4393714835365918045""","[0.013646, 0.013646, … 0.013646]","[0.005886, 0.005886, … 0.005886]","[0.011229, 0.011229, … 0.011229]","[0.00545, 0.00545, … 0.00545]","[0.01104, 0.01104, … 0.01104]","[0.003369, 0.003369, … 0.003369]","[0, 0, … 0]","[0.0, 0.0, … 0.0]"
"""1573919593896895308""","[0.013646, 0.013646, … 0.013646]","[0.018963, 0.018963, … 0.018963]","[0.011229, 0.011229, … 0.011229]","[0.016248, 0.016248, … 0.016248]","[0.01104, 0.01104, … 0.01104]","[0.019489, 0.019489, … 0.019489]","[0, 0, … 1]","[0.0, 0.0, … 0.253521]"


- A list of purchasing visitors is created to select recommendations for 

In [20]:
purchasing_visitors=encoded_df.filter(pl.col('transactionId') != 0).select(pl.col('fullVisitorId')).unique()
purchasing_visitors.write_csv("../data/purchasing_visitors.csv")

- Recommendations and their accuracy metrics are calculated

In [21]:
recommendation_eval_pipeline("2981325201816358942", user_profiles_, encoded_df, df)

0.25

In [22]:
recommendation_pipeline("2981325201816358942", user_profiles_, encoded_df, df)

['Waze', 'Apparel', 'Apparel', 'Bottles', 'Drinkware']

### Data Pre-processing Pipeline

In [23]:
def preprocessing_pipeline(df: pl.DataFrame, active_user: str) -> pl.DataFrame:
    """
    Args:
        df : pl.DataFrame

    Returns:
        pl.DataFrames: pre-processed dataframe
    """
    df = df.filter(~pl.col('v2ProductCategory').str.contains('origCatName'))
    df = df.with_columns(pl.col("fullVisitorId").cast(str))
    df = df.with_columns(pl.col('date').str.strptime(pl.Date, format="%Y%m%d"))
    encoded_df_ = target_encoding(df)
    
    user_profiles_ = user_profiles(encoded_df_)

    rec = recommendation_pipeline(active_user, user_profiles_, encoded_df, df)
    
    return rec

In [24]:
preprocessing_pipeline(df,"2981325201816358942")

['Waze', 'Headgear', 'Apparel', 'Bottles', 'Drinkware']

# Conclusions 
- The data has been encoded to allow recommendations to be calculated 
- Recommendations and their accuracy are calculated for a respective user
- A recommendation pipeline has been created 